In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import os

In [32]:
raw_data = pd.read_csv('empresasEafit.csv')
raw_data.rename(columns=lambda x: x.strip(), inplace=True)
raw_data = raw_data[~raw_data['Macrosector'].isin(['0', 'No', 'No informa', 'SI', 'Si'])]
raw_data.head()

,Razón social,Macrosector,"Tipo de propiedad (Privada, Pública, Mixta)",¿Es empresa familiar? Si/No,¿Cotiza en bolsa? Si/No,¿Multinacional? Si/No,Año de fundación,País de base principal (nacionalidad),Ingresos operacionales,Variable,Valor,Bloque,Nombre Pilar,Ponderación Materialidad,Valoración,valoracionPonderada
0,Aliar,Agropecuario,Privada,No,No,No,2007,Colombia,0.97802211500000003,¿Cuál es el porcentaje de proveeduría local?,No informa,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",14%,0.000000,0.000000
1,Alimentos Carnicos (Grupo Nutresa),Manufactura,Privada,No,Si,Si,1920,Colombia,2.844632077,¿Cuál es el porcentaje de proveeduría local?,0.80000000000000004,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",11%,84.210526,8.864266
2,Alpina,Manufactura,Privada,No,No,Si,1945,Colombia,2.1764613960000001,¿Cuál es el porcentaje de proveeduría local?,0.91000000000000003,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",11%,95.789474,10.083102
3,Amarilo,Construcción,Privada,Si,No,No,1993,Colombia,2.1863011939999999,¿Cuál es el porcentaje de proveeduría local?,0.53000000000000003,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",14%,53.000000,7.571429
4,ARA,Comercio,Privada,No,No,Si,2012,Colombia,13.83118268,¿Cuál es el porcentaje de proveeduría local?,0,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",12%,0.000000,0.000000


In [7]:
def save_chart_as_html(fig, filename):
    """
    Guarda una figura de Plotly como un archivo HTML autocontenido en la carpeta 'charts'.
    """
    # Generar el HTML del gráfico
    html1 = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
    
    # Guardar el archivo
    filepath = os.path.join('charts', filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(html1)

# Function to wrap long text labels
def wrap_text(text, max_length=25):
    """
    Wrap text to multiple lines if it exceeds max_length characters.
    """
    if len(text) <= max_length:
        return text
    
    words = text.split()
    lines = []
    current_line = []
    current_length = 0
    
    for word in words:
        # Check if adding this word would exceed the limit
        if current_length + len(word) + len(current_line) > max_length and current_line:
            lines.append(' '.join(current_line))
            current_line = [word]
            current_length = len(word)
        else:
            current_line.append(word)
            current_length += len(word)
    
    if current_line:
        lines.append(' '.join(current_line))
    
    return '<br>'.join(lines)


# **Parte 1: Panorama General del Desempeño Corporativo**

## 1. Radar de Competitividad por Pilar y Macrosector

### Manejo de datos

In [8]:
# Encadenar las operaciones de pandas para simplificar el proceso:
# 1. Agrupar por Macrosector y Nombre Pilar, calculando la media.
radar_final_df = (
    raw_data.groupby(['Macrosector', 'Nombre Pilar'])['valoracionPonderada']
    .mean()
    .reset_index()
    .pivot_table(
        index='Nombre Pilar',
        columns='Macrosector',
        values='valoracionPonderada'
    )
    .fillna(0)
    .reset_index()
    .rename(columns={'Nombre Pilar': 'category'})
)

categories = radar_final_df['category'].tolist()
macrosectores = radar_final_df.drop(columns='category').columns.tolist()

# Wrap category labels
wrapped_categories = [wrap_text(cat) for cat in categories]

fig = go.Figure()

# Create color palette for different macrosectors
colors = px.colors.qualitative.Set1[:len(macrosectores)]

for i, macrosector in enumerate(macrosectores):
    values = radar_final_df[macrosector].values
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=wrapped_categories,  # Use wrapped categories
        fill='toself',
        name=macrosector,
        line=dict(color=colors[i % len(colors)]),
        fillcolor=colors[i % len(colors)],
        opacity=0.6
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, max(radar_final_df.drop(columns='category').max()) * 1.1],  # Dynamic range
            tickmode='linear',
            tick0=0,
            dtick=5
        ),
        angularaxis=dict(
            tickmode='array',
            tickvals=list(range(len(categories))),
            ticktext=wrapped_categories,  # Use wrapped categories
            tickfont=dict(size=10),  # Smaller font for wrapped text
            rotation=0  # Keep labels horizontal for better readability
        )
    ),
    showlegend=True,
    title={
        'text': '1. Radar de Competitividad por Pilar y Macrosector',
        'x': 0.5,
        'xanchor': 'center'
    },
    width=900,  # Increased width to accommodate wrapped labels
    height=700  # Increased height to accommodate wrapped labels
)
fig.show()
save_chart_as_html(fig, '01_radar_macroeconomic.html')


## 2. Mapa de calor de Desempeño Empresarial

### Manejo de datos

In [28]:
# 1. Pivotar la tabla para agregar los valores.
#    - Se agrupa por 'Razón social' y 'Nombre Pilar'.
#    - Se calcula la SUMA de 'valoracionPonderada' como indica la guía.
#    - Se rellenan los valores nulos con 0.
heatmap_pivot = raw_data.pivot_table(
    index='Razón social',
    columns='Nombre Pilar',
    values='valoracionPonderada',
    aggfunc='sum',
    fill_value=0
)

heatmap_pivot.sort_values(by='Razón social', inplace=True)
wrapped_pilars = [wrap_text(col) for col in heatmap_pivot.columns]
# Crear mapa de calor con Plotly
heatmap_fig = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=wrapped_pilars,
    y=heatmap_pivot.index,
    colorscale='blues',
    colorbar=dict(title='Valoración Ponderada')
))

heatmap_fig.update_layout(
    title='2. Mapa de Calor de Desempeño Empresarial',
    xaxis_title='Pilares',
    yaxis_title='Empresas',
    width=1000,  # Ajustar el ancho del gráfico
    height=800  # Ajustar la altura del gráfico
)

heatmap_fig.show()
save_chart_as_html(heatmap_fig, '02_heatmap_performance.html')

## 3. Diagrama de Violín de la Dispersión del Desempeño Sectorial

### Manejo de datos

In [34]:
# 1. Calcular el puntaje total (suma de valoracionPonderada) para cada empresa.
total_scores = raw_data.groupby(['Razón social', 'Macrosector'])['valoracionPonderada'].sum().reset_index()
total_scores.rename(columns={'valoracionPonderada': 'puntaje_total'}, inplace=True)

# 2. Eliminar Razón social y limpiar Macrosector '0' 'No' 'No informa' 'SI' 'Si'
total_scores.drop(columns='Razón social', inplace=True)


# 3. Crear un violin plot para cada Macrosector.
violin_fig = px.violin(total_scores, y='puntaje_total', x='Macrosector', box=True, points='all',
               color='Macrosector', title='3. Violin Plot de Puntajes por Macrosector',
               labels={'puntaje_total': 'Puntaje Total', 'Macrosector': 'Macrosector'})
violin_fig.update_traces(meanline_visible=True)
violin_fig.update_layout(
    xaxis_title='Macrosector',
    yaxis_title='Puntaje Total',
    width=900,  # Ajustar el ancho del gráfico
    height=700  # Ajustar la altura del gráfico
)

violin_fig.show()
save_chart_as_html(violin_fig, '03_violin_plot_performance.html')

## 4. Treemap de Impacto: Sostenibilidad y Peso Económico

### Manejo de datos